In [126]:
import pandas as pd
import numpy as np
import torch
import networkx as nx
import cobra
import matplotlib.pyplot as plt

import sys
sys.path.append("../src/")
import GEMtoGRAPH as gg

In [127]:
model = cobra.io.load_json_model('redYeast_ST8943_fdp1.json')
S = cobra.util.array.create_stoichiometric_matrix(model, array_type='DataFrame')
S.shape

(300, 373)

# MFG

#### Load TFA fluxes

In [128]:
# load tfa fluxes and send them to graph construction functions
tfa = pd.read_csv('fluxes_for_graph.csv', index_col=0)
tfa = tfa.head(1)

zero_flux = [col for col in tfa.columns if (tfa[col] == 0).all()]

print('Zero flux reactions:',len(zero_flux))

tfa.drop(columns=zero_flux, inplace=True)
print("TFA fluxes:", tfa.shape[1])

# For _reverse reactions we should change the sign of the flux to negative
for col in tfa.columns:
    if '_reverse' in col: tfa[col] = -tfa[col]


tfa.rename(columns={col: col.split("_reverse_")[0] for col in tfa.columns}, inplace=True)

tfa_flux = tfa.iloc[0].values
tfa_flux = pd.DataFrame(columns=['fluxes'], data=tfa_flux)
tfa_flux.index = S.columns

Zero flux reactions: 373
TFA fluxes: 373


### Create MFG Graph

In [129]:
M, S_2m, G = gg.MFG(S, model, tfa_flux)

# nodes: 746 
# edges: 6157


In [130]:
# Remove isolated nodes from G
isolated_nodes = list(nx.isolates(G))
G.remove_nodes_from(isolated_nodes)

print("# nodes:", G.number_of_nodes(), "\n# edges:", G.number_of_edges())

# nodes: 373 
# edges: 6157


## Read ORACLE's data

In [131]:
sigma = pd.read_csv('saturations.csv', index_col=0)
gamma = pd.read_csv('gamma.csv', index_col=0)
vmax = pd.read_csv('Vmax_matrix.csv', index_col=0)

gamma = gamma.head(1)
sigma = sigma.head(1)
vmax = vmax.head(1)

In [132]:
# get the reactions that are the reversible version
rev_rxn = []
for node in list(G.nodes()):
    if node.split("?")[0] == 'rev': rev_rxn.append(node.split("?")[1])

# rename the reactions of gamma; if it's the reversible one add rev? to the column name
for col in gamma.columns:
    if col in rev_rxn: gamma.rename(columns={col:'rev?'+col}, inplace=True)

In [133]:
listA = list(G.nodes())
listB = gamma.columns

print('In Graph but not in gamma:', [item for item in listA if item not in listB])
print()
print('In gamma but not in Graph:', [item for item in listB if item not in listA])

In Graph but not in gamma: ['EX_lac__D_e', 'EX_mal__L_e', 'EX_akg_e', 'EX_2phetoh_e', 'EX_acald_e', 'EX_ac_e', 'EX_gam6p_e', 'EX_co2_e', 'EX_cit_e', 'EX_etoh_e', 'EX_fum_e', 'EX_gly_e', 'EX_gcald_e', 'EX_glx_e', 'EX_id3acald_e', 'EX_ala__L_e', 'EX_asn__L_e', 'EX_asp__L_e', 'EX_cys__L_e', 'EX_glu__L_e', 'EX_gln__L_e', 'EX_phe__L_e', 'EX_ser__L_e', 'EX_trp__L_e', 'EX_tyr__L_e', 'EX_oaa_e', 'EX_pacald_e', 'EX_pyr_e', 'EX_succ_e', 'EX_ind3eth_e', 'EX_h2o_e', 'EX_g6p_e', 'EX_g1p_e', 'EX_2pg_e', 'EX_pser__L_e', 'EX_ppi_e', 'EX_pep_e', 'EX_cbp_e', 'EX_6pgc_e', 'EX_3pg_e', 'EX_cmp_e', 'GROWTH', 'EX_ccm_e', 'EX_pca_e', 'rev?EX_nh4_e', 'rev?EX_glc__D_e', 'rev?EX_h_e', 'rev?EX_fe2_e', 'rev?EX_o2_e', 'rev?EX_pi_e', 'rev?EX_k_e', 'rev?EX_na1_e', 'rev?EX_so4_e', 'rev?EX_cl_e', 'rev?EX_cu2_e', 'rev?EX_mn2_e', 'rev?EX_zn2_e', 'rev?EX_mg2_e', 'rev?EX_ca2_e']

In gamma but not in Graph: []


#### Add `gamma` values as Graph node features

In [134]:
for node in gamma.columns:
    try:
        G.nodes[node]['gamma'] =  gamma[node].values[0]
    except KeyError:
        pass

no_gamma_nodes = [node for node, data in G.nodes(data=True) if not data]

for node in no_gamma_nodes: G.nodes[node]['gamma'] = np.nan

### We have the `Networkx` Graph G

In [135]:
G.number_of_nodes(), G.number_of_edges()

(373, 6157)

### Add more node features using data from GEM

### Create a _node features_ matrix

In [153]:
df = pd.DataFrame(index=list(G.nodes()), columns=['compartements', 'metabolites', 'num_of_mets'])

for rxn in df.index:
    if 'rev?' in rxn: rxn = rxn.split("?")[1]
    else: rxn = rxn

    metabolites = []
    for m in model.reactions.get_by_id(rxn).metabolites: metabolites.append(m.id)
  
    metabolites = "|".join(metabolites)
    compartements = "|".join(list(model.reactions.get_by_id(rxn).compartments))

    # compartements = list(model.reactions.get_by_id(rxn).compartments)
    
    num_of_mets = len(model.reactions.get_by_id(rxn).metabolites)
    new_row = [compartements, metabolites, num_of_mets]

    df.loc[rxn] = new_row

df.head(3)

,compartements,metabolites,num_of_mets
D_LACDcm,c|m,ficytc_m|focytc_m|lac__D_c|pyr_c,4
D_LACDm,m,ficytc_m|focytc_m|lac__D_m|pyr_m,4
L_LACD2cm,c|m,ficytc_m|focytc_m|lac__L_c|pyr_c,4


In [144]:
# LABEL ENCODING to every compartement and metabolite
from sklearn.preprocessing import LabelEncoder

def enc_for_every():
    COBRA_METABOLITES = pd.DataFrame([m.id for m in model.metabolites])
    COBRA_METABOLITES.rename(columns = {0:'id'}, inplace = True)
    COBRA_METABOLITES['enc'] = LabelEncoder().fit_transform(COBRA_METABOLITES['id'])

    COBRA_COMPARTEMETNS = pd.DataFrame(list(model.compartments.keys()))
    COBRA_COMPARTEMETNS.rename(columns= {0:'id'}, inplace=True)
    COBRA_COMPARTEMETNS['enc'] = LabelEncoder().fit_transform(COBRA_COMPARTEMETNS['id'])

    for row in range(len(df)):

        c = df.iloc[row]['compartements']
        m = df.iloc[row]['metabolites']

        try:
            df.loc[df.index[row], 'compartements'] = ([dict(COBRA_COMPARTEMETNS.values).get(item, item) for item in c])
            df.loc[df.index[row], 'metabolites'] = ([dict(COBRA_METABOLITES.values).get(item, item) for item in m])
        except TypeError:
            pass

In [156]:
# Encoding for c|m -> 0, ... 
df['compartements'] = LabelEncoder().fit_transform(df['compartements'])

In [157]:
for node in list(G.nodes()):
    if 'rev?' in node: rxn = node.split("?")[1]
    else: rxn = node

    nx.set_node_attributes(G, {node: {'compartements':df.loc[rxn]['compartements']}})
    # nx.set_node_attributes(G, {node: {'metabolites':df.loc[rxn]['metabolites']}})
    nx.set_node_attributes(G, {node: {'num_of_mets':df.loc[rxn]['num_of_mets']}})

In [170]:
from operator import itemgetter

def orderDict(x: dict, desc=True):
    return sorted(x.items(), key=itemgetter(1), reverse=desc)

indg = orderDict(nx.in_degree_centrality(G))

ccen = orderDict(nx.closeness_centrality(G))

betcen = orderDict(nx.betweenness_centrality(G))

pgrk = orderDict(nx.pagerank(G))

hubs, autr = nx.hits(G)

hubs = orderDict(hubs)
autr = orderDict(autr)

col_names = ['in-degree', 'closeness', 'betweness', 'page rank', 'autr', 'hubs']

graphStats = pd.DataFrame(columns=col_names)

graphStats['in-degree'] = [n for n, v in indg]
graphStats['closeness'] = [n for n, v in ccen]
graphStats['betweness'] = [n for n, v in betcen]
graphStats['page rank'] = [n for n, v in pgrk]
graphStats['autr'] = [n for n, v in autr]
graphStats['hubs'] = [n for n, v in hubs]

# print('Top:\n')
# display(df.head(10))
# print('Bottom:\n')
# display(df.tail(10))


/home/comex/Desktop/python-envs/main-bio/lib/python3.9/site-packages/networkx/algorithms/link_analysis/hits_alg.py:78: FutureWarning: adjacency_matrix will return a scipy.sparse array instead of a matrix in Networkx 3.0.
  A = nx.adjacency_matrix(G, nodelist=list(G), dtype=float)


## Node2Vec 

In [ ]:
G_labels = nx.convert_node_labels_to_integers(G)

from karateclub import Node2Vec

" Perform node embedding using Node2Vec "
N2vec_model = Node2Vec(walk_number=10, walk_length=80,p=0.9 ,q=0.1,dimensions=12)
N2vec_model.fit(G_labels)
N2Vec_embedding = N2vec_model.get_embedding()
print('Embedding array shape (nodes x features):',N2Vec_embedding.shape )

df_embedding = pd.DataFrame(index=list(G.nodes), data=N2Vec_embedding)
df_embedding.head(3)

#### Clustering based on embeddings

In [ ]:
from sklearn.cluster import KMeans

kmeans = KMeans(n_clusters=4, random_state=0).fit(df_embedding)

results = pd.DataFrame(columns=['cluster'], index=df_embedding.index)
results['cluster'] = kmeans.labels_
results

## Networkx to Torch Geometric

In [159]:
import torch
from torch_geometric.utils.convert import from_networkx

device = 'cuda' if torch.cuda.is_available() else 'cpu'

data = from_networkx(G, group_node_attrs=all, group_edge_attrs=all)
print(data)
data.num_nodes ,data.num_edges

Data(edge_index=[2, 6157], x=[373, 3], edge_attr=[6157, 1])


(373, 6157)